In [1]:
import requests
import time
from bs4 import BeautifulSoup as bs

In [56]:
def get_info(url, headers):
    r = requests.get(url, headers)
    #Convert to a beautifulsoup object
    soup = bs(r.content)
    
    price = soup.find(class_='offer__price').get_text(' ', strip=True).replace('\xa0', '')
    price = price[:-2]
    
    #////////////////////
    roomslst = soup.find(class_='offer__advert-title').get_text(' ', strip=True).replace('\xa0', '')
    num_room = roomslst[0]
    
    all_info2 = []
    all_info3 = []
    
    ch_lst = []
    sign_pos = 2
    zone1 = soup.find(class_='offer__location offer__advert-short-info').get_text()
    #for ind, ch in enumerate(zone1):
        #ch_lst.append(ch)
        
    #zone1 = zone1[:sign_pos+1]
    
    short_data = soup.find_all(class_='offer__advert-short-info')
    for a in short_data:
        all_info3.append(a.get_text())
    
    short_data2 = soup.find_all(class_='offer__parameters')
    for a in short_data2:
        all_info2.append(a.get_text())
    
    lens = len(all_info3)
    
    ai2 = [i.replace(' ', '') for i in all_info2]
    ai2 = [i.replace('\n\n\n', ', ') for i in ai2]
    ai2 = [i.replace('\n\n', '') for i in ai2]
    ai2 = [i.replace('\n', ' ') for i in ai2]
    another_list = all_info3[-(lens-1):] + ai2
    
    final_list = []
    split_str = another_list[-1].split(', ')
    main_list = another_list[:-1]
    
    for i in main_list:
        if i == "монолитный" or i == "панельный" or i == "кирпичный":
            building_type = i
            break
        building_type = '-'
    
    eng_abc = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","Y","Z"]
    rus_abc = ["Ц","У","К","Е","Н","Г","Ш","Щ","З","Х","Ф","Ы","В","А","П","Р","О","Л","Д","Ж","Э","Я","Ч","С","М","И","Т","Б","Ю","Ё","«"]
    
    for i in main_list:
        if i[0] in eng_abc or i[0] in rus_abc:
            zone = i
            break
        zone = '-'
    for i in main_list:
        if i[:2] == '20' or i[:2] == '19':
            if int(i) >= 1970 and int(i) <= 2024:
                year = int(i)
    n_floor = []
    total_floor = []
    n_floor = [int(i[:2]) for i in main_list if i[2:4] == 'из']
    total_floor = [int(i[-2:]) for i in main_list if i[2:4] == 'из']
    
    if n_floor == []:
        n_floor.append('-')
    if total_floor == []:
        total_floor.append('-')
        
    for i in main_list:
        if 'Площадь кухни' in i:
            for ind, j in enumerate(i):
                if j == "П":
                    sqrft_kchn = i[ind+16:]
                    
            break
        else:
            sqrft_kchn = 0  
            
    sqrft = [int(i[:2]) for i in main_list if 'м²' in i]
    
    
    if 'паркинг' in another_list:
        parking = 1
    elif 'Парковка паркинг' in split_str:
        parking = 1
    else:
        parking = 0
        
    if 'черновая отделка' in another_list:
        rough_finish = 1
    elif 'Состояние черновая отделка' in split_str:
        rough_finish = 1
    else:
        rough_finish = 0
        
    ceiling = []
    ceiling = [i[-4:] for i in split_str if 'Потолки' in i and '.' in i]
    if ceiling == []:
        ceiling = [i[-2:] for i in split_str if 'Потолки' in i]
    if ceiling == []:
        ceiling.append('-')
        
    if 'Санузел 2с/уиболее' in split_str:
        toilet = 1
    else:
        toilet = 0
        
        
        
        
    final_list.append(building_type)
    final_list.append(zone1)
    final_list.append(year)
    final_list.append(n_floor[0])
    final_list.append(total_floor[0])
    final_list.append(sqrft[0])
    final_list.append(sqrft_kchn)
    final_list.append(ceiling[0])
    final_list.append(toilet)
    final_list.append(parking)
    final_list.append(rough_finish)
    final_list.append(int(num_room))
    final_list.append(price)
    
        
    return final_list

In [57]:
print(get_info('https://krisha.kz/a/show/682844355', {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}))

['кирпичный', '\nАстана, Есильский р-н\nпоказать на карте\n', 2021, 7, 12, 59, '8 м²', '3м', 0, 0, 0, 2, '30900000']


In [210]:
print(get_info('https://krisha.kz/a/show/681794518'))

['панельный', '-', 1990, 4, 9, 66, '8.5 м²', '2.7м', 0, 0, 0, 3, 31000000]


In [211]:
print(get_info('https://krisha.kz/a/show/682225715'))

['кирпичный', 'Mechta', 2022, 7, 10, 37, 0, '3м', 0, 0, 0, 1, 23500000]


In [58]:
def ad_conn(url, headers):
    
    base_path = 'https://krisha.kz'
    r = requests.get(url, headers)
    soup = bs(r.content)
    ads = soup.find_all(class_='a-card__title')
    
    links = []
    for ad in ads:
        links.append(ad.get('href'))
    
    dataset = []
    for index, link in enumerate(links):
        try:
            relative_path = link
            full_path = base_path + relative_path
            dataset.append(get_info(full_path, headers))
        except Exception as e:
            continue
            
    return dataset

In [63]:
print(ad_conn('https://krisha.kz/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1'))

[['монолитный', 'Мерей Интерконти', 2007, 2, 20, 45, 0, '3м', 0, 1, 0, 2, 20500000], ['монолитный', 'Медина', 2019, 8, 13, 41, 0, '2.7м', 0, 1, 0, 1, 15500000], ['монолитный', 'Koktal park', 2022, 1, 6, 25, 0, '2.8м', 0, 0, 1, 1, 8200000], ['монолитный', 'Гранд Астана Элит', 2011, 3, 21, 10, 0, '3м', 1, 1, 0, 3, 46000000], ['кирпичный', 'ЖК Баскару 2, 3, 4', 2022, '-', '-', 32, 0, '2.7м', 0, 1, 0, 1, 15000000], ['монолитный', 'Кок жайлау', 2019, 8, 10, 40, 0, '-', 0, 1, 0, 2, 23999000], ['монолитный', 'Алмалы', 2016, '-', '-', 62, 0, '-', 0, 1, 0, 2, 33500000], ['монолитный', 'Angleterre', 2019, 5, 9, 38, '11 м²', '3м', 0, 0, 0, 1, 18500000], ['кирпичный', 'Жети Жол', 2015, '-', '-', 42, 0, '-', 0, 0, 0, 2, 18900000], ['кирпичный', 'Шыгыс', 2021, 4, 7, 76, '9 м²', '-', 0, 0, 0, 3, 32500000]]


In [59]:
def pg_conn(url, headers, x):
    r = requests.get(url, headers)
    soup = bs(r.content)
    pgs = soup.find_all(class_='paginator__btn')
    
    link = pgs[x+1].get('href')
    
    return link

In [42]:
print(pg_conn('https://krisha.kz/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1', 0))

['/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=2', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=3', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=4', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=5', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=6', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=7', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=8', None, '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=1000', '/prodazha/kvartiry/astana/?das[_sys.fromAgent]=1&das[who]=1&page=2']


In [80]:
import csv

url = "https://krisha.kz/prodazha/kvartiry/astana/?page=975"
dataset = []
base_path = 'https://krisha.kz'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
y = 0
for x in range(25):
    if y % 3 == 0:
        time.sleep(40)
    dataset.append(ad_conn(url, headers))
    url = base_path + pg_conn(url, headers, 6)
    y+=1
        

datasetto = []
if dataset != []:
    for i in dataset:
        for j in i:
            datasetto.append(j)

    header = ['building_type','zone','year','n_floor','total_floor','area','kchn_area', \
              'ceiling','toilet','parking','rough_finish','num_room','price']

    with open('Df1000.csv', 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i in range(len(datasetto)):
            building_type = datasetto[i][0]
            zone = datasetto[i][1]
            year = datasetto[i][2]
            n_floor = datasetto[i][3]
            total_floor = datasetto[i][4]
            area = datasetto[i][5]
            kchn_area = datasetto[i][6]
            ceiling = datasetto[i][7]
            toilet = datasetto[i][8]
            parking = datasetto[i][9]
            rough_finish = datasetto[i][10]
            num_room = datasetto[i][11]
            price = datasetto[i][12]
        
            data = [building_type, zone, year, n_floor, total_floor, area, kchn_area, ceiling, toilet, parking, rough_finish, num_room, price]
            writer.writerow(data)
            
print(len(datasetto))

492


In [10]:
print(len(dataset[106]))

20


In [12]:
dataset_100 = dataset

final_list.append(building_type)
    final_list.append(zone)
    final_list.append(year)
    final_list.append(n_floor[0])
    final_list.append(total_floor[0])
    final_list.append(sqrft[0])
    final_list.append(sqrft_kchn)
    final_list.append(ceiling[0])
    final_list.append(toilet)
    final_list.append(parking)
    final_list.append(rough_finish)
    final_list.append(int(num_room))
    final_list.append(price)

In [14]:
n_records = 0
dataset_100_new = []
for i in dataset_100:
    for j in i:
        dataset_100_new.append(j)
print(len(dataset_100_new))

2092


In [15]:
import csv

header = ['building_type','residential_complex','year','n_floor','total_floor','area','kchn_area', \
          'ceiling','toilet','parking','rough_finish','num_room','price']

with open('Dataset100.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(len(dataset_100_new)):
        building_type = dataset_100_new[i][0]
        residential_complex = dataset_100_new[i][1]
        year = dataset_100_new[i][2]
        n_floor = dataset_100_new[i][3]
        total_floor = dataset_100_new[i][4]
        area = dataset_100_new[i][5]
        kchn_area = dataset_100_new[i][6]
        ceiling = dataset_100_new[i][7]
        toilet = dataset_100_new[i][8]
        parking = dataset_100_new[i][9]
        rough_finish = dataset_100_new[i][10]
        num_room = dataset_100_new[i][11]
        price = dataset_100_new[i][12]
        
        data = [building_type, residential_complex, year, n_floor, total_floor, area, kchn_area, ceiling, toilet, parking, rough_finish, num_room, price]
        writer.writerow(data)

In [16]:
import pandas as pd

df = pd.read_csv('Dataset100.csv')

print(df.head())
print(len(df))

  building_type residential_complex  year n_floor total_floor  area kchn_area  \
0    монолитный         Столичный 2  2023       -           -    65         0   
1     кирпичный                Yrys  2024       -           -    40         0   
2    монолитный              Shabyt  2023       -           -    41         0   
3    монолитный                   -  1983       4           5    72      9 м²   
4    монолитный          Esil Tower  2022       -           -    47     13 м²   

  ceiling  toilet  parking  rough_finish  num_room        price  
0       -       0        0             0         2  от 26376000  
1       -       0        0             0         1   от 9796800  
2       -       0        0             0         1  от 22888540  
3       -       0        0             0         4     34000000  
4    2.9м       0        1             0         1     35000000  
2092
